# Hybrid Demucs from Colab

In [1]:
%%capture

!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs
!pip install basic-pitch --upgrade

In [27]:
from argparse import ArgumentParser
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO
from os.path import dirname, abspath
from basic_pitch.inference import predict
from basic_pitch import ICASSP_2022_MODEL_PATH
import numpy as np
import librosa

model and hyperparamters

In [ ]:
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

functions for source separation

In [3]:
def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out


def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

# separate all song in directory passed in
def separate_dir(inp=None, outp=None):
    inp = inp 
    outp = outp 
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {inp}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def separate(inp=None, outp=None):
    inp = inp 
    outp = outp 
    cmd = ["demucs", "-o", str(outp)]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]

    print("Going to separate file:")
    print(inp)
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + [inp], stdout=sp.PIPE, stderr=sp.PIPE)

    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


input mp3 path and output path

In [4]:
in_path = '/content/thunderstruck.mp3'
out_path = '/content/'

generate stem and midi from stem

In [6]:
# other.mp3 will have the guitar stem
separate(in_path, out_path)

# midi : pretty_midi.PrettyMIDI object
# note_events: A list of note event tuples (start_time_s, end_time_s, pitch_midi, amplitude, bends)
# multiple notes can have the same start time
outfile_path = out_path + 'htdemucs/' + in_path.split('/')[-1].split('.mp3')[0] + '/other.mp3'

model_output, midi_data, note_events = predict(outfile_path)

Predicting MIDI for /content/htdemucs/thunderstruck/other.mp3...


note output

In [33]:
def pitch_to_note(note_events):
    ret_list = []
    for event in note_events:
        l = list(event)
        note = librosa.midi_to_note(event[2], unicode=False)  # event[2] is pitch
        l[2] = note
        ret_list.append(l)
    return ret_list


# sort note events according to start time 
sorted_events = sorted(note_events, key=lambda x: x[0])  # x[0] is start time

# midi pitch to note
# each event is now a list (previously tuple)
music_note_events = pitch_to_note(sorted_events)

In [ ]:
len(music_note_events)

In [35]:
music_note_events[0]

[0.4992290249433107,
 0.7314285714285714,
 'B3',
 0.45735344,
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 0, 0, 0, 0, 1, 1, 1, 1]]